# nb 07: change and use cobot ee-tools


**This is the continuation of the previous tutorial notebooks. 
Please understand how to read data and send simple commands from the cobot before continuing here.
You can also find additional information about underlying docus and startup procedures**

The code below requires:

- ``hrr_msgs >= 0.1.4``
- ``hrr_cobot_robot >= 0.2.1`` (0.1.4 is partially okay, but digital I/O requires ``0.2.1`` and newer)


## Imports / Packages needed

In [2]:
# packages needed for this tutorial
from pathlib import Path

# math and robotics
import numpy as np
import spatialmath as sm
import quaternion
from tqdm.notebook import tqdm, trange

# plotting 
import seaborn as sns
import matplotlib.pylab as plt

# ROS
import rospy

# hrr-cobot packages
import hrr_cobot_robot
import hrr_common
import hrr_controllers

# set printing and plotting options    
np.set_printoptions(precision=5, suppress=True)
sns.set_theme('notebook')
%matplotlib notebook

**connect to ROS**

optionally set the ```ROS_IP``` and ```ROS_MASTER_URI``` directly form this notebook, as
the setup above spawns the ros-setup from the real-time client, i.e. ```hrrN3511rt2004.lsr.ei.tum.de```
for a local setup, replace the above by: 

```ipython
%env ROS_MASTER_URI=http://localhost:11311
```

In [2]:
def get_hostname_and_IP():
    import socket
    with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s:
        s.connect(("8.8.8.8", 80))
        return socket.gethostname(), s.getsockname()[0]

_host, _ip = get_hostname_and_IP()
%env ROS_MASTER_URI=http://127.0.0.1:11311
# %env ROS_MASTER_URI=http://hrrcobotLinux54:11311
print(f"\t\033[1m\033[4m!!!please check that the IPs below is identical!!!\033[0m"+
      f"\n\thost {_host} has IP {_ip}, which should be identical to",end="\nvs:\t")
%env ROS_IP=$_ip

env: ROS_MASTER_URI=http://127.0.0.1:11311
	!!!please check that the IPs below is identical!!!
	host hrrN3511rt2004 has IP 192.168.2.34, which should be identical to
vs:	env: ROS_IP=192.168.2.34


In [3]:
rospy.init_node("hrr_cobot_tutorial07")

## Available Tools

the new launching pipeline for setting a dedicated HRR-cobot tool is given as

```bash
roslaunch hrr_cobot_robot hrr_cobot_hw.launch tool_name:="shaftgrinder"
```

where ```tool_name``` can be set toGRGR one of the following:

* "wsg_50"
* "wsg_50_dsa"
* "shaftgrinder"
* "screwdriver"
* "vacuum"

this will adjust / set the ROS-URDF in RVIZ as shown below:

![Shaftgrinder Cobot Rviz](media/shaftgrinder_cobot.png)

### Read tool data 

This allows us to read the pose of the tool-tip from ROS using the tf-transformations

We show this via the ``TfHandler`` from the ``hrr_common`` package below.

In [4]:
from hrr_common.ros_utils import TfHandler

tf = TfHandler(add=False)

In [4]:
tf.T_A_B(A='wsg_50', B='base_link')

(array([0., 0., 0.]), quaternion(1, 0, 0, 0))

In [6]:
tf.A_vector(B_vector=np.r_[0., 0., 1.], frame_B="base_link", frame_A="shaftgrinder_tip")

array([0., 0., 1.])

In [7]:
p, q = tf.T_A_B('shaftgrinder_tip', 'base_link')
T_B_tool = sm.SE3(*p)
T_B_tool.A[:3, :3] = quaternion.as_rotation_matrix(q)
T_B_tool

   1         0         0         0         
   0         1         0         0         
   0         0         1         0         
   0         0         0         1         


## commanding the robot in tool frame

Given this setup, we either can manually create the desired end-effector velocity
to be sent to the COMAU robot or use the ``sensor-track-velocity`` controller to 
apply the set velocity in a dedicated reference frame. 

### send velocity commands in tool-frame

In [ ]:
# rospy.set_param("~cmd_arduino", False)

In [5]:
cobot = hrr_cobot_robot.HrrCobotControl.from_ros(cobot_prefix="/hrr_cobot/")


[rosout] [/hrr_cobot_tutorial07] Initialize MATLAB-Motion planner @ /home/hrr_cobot/_ros/hr_recycler_ws/src/planner/src
[rosout] [/hrr_cobot_tutorial07] connected to Matlab
The value of the ROS_IP environment variable, 129.187.147.54, will be used to set the advertised address for the ROS node.
Initializing global node /matlab_global_node_89461 with NodeURI http://129.187.147.54:36427/
[rosout] [/hrr_cobot_tutorial07] MATLAB-Initialization done


pybullet build time: May  8 2021 05:48:13


[rosout] some functions are not yet compiled. Expect delays upon first call


In [7]:
sns_trk_vel_cmd = hrr_controllers.SnsTrkCmd.from_ros(cobot_prefix="/hrr_cobot/")

In [8]:
cobot = hrr_cobot_robot.HrrCobotControl.from_ros("/hrr_cobot")

[rosout] [/hrr_cobot_tutorial07] Initialize MATLAB-Motion planner @ /home/hrr_cobot/_ros/hr_recycler_ws/src/planner/src


[rosout] failed to initialize Matlab-connection due to: Unable to connect to MATLAB session 'MATLAB_112115'.


[rosout] some functions are not yet compiled. Expect delays upon first call


### Use the sensor-track velocity command interface directly

we apply a similar command as in tutorial 2 but apply the velocity along 
the $z$-axis of the ``shaftgrinder_tip`` frame.

In [9]:
sns_trk_vel_cmd.activate()

In [13]:
r = rospy.Rate(50)
v_test = np.zeros(6)
for t in trange(200):
    v_test[2] = 1e-2 * np.sin(t/ 100.0 * 2 * np.pi)
#     v_test[1] = 1e-2 * np.sin(t/ 100.0 * 2 * np.pi)
#     v_test[2] = 5e-3 * np.sin(t/ 100.0 * 2 * np.pi)
    sns_trk_vel_cmd.update_cmd(v_test)
    r.sleep() 
sns_trk_vel_cmd.stop()

  0%|          | 0/200 [00:00<?, ?it/s]

## Tool-changing Utility

### Use the ``cobot`` interface

Similarly we can also use the ``HrrCobotControl`` handle via the ``update`` function

In [ ]:
?cobot.update

In [ ]:
cobot.init_sns_vel()

In [ ]:
v_test = np.zeros(6)
for t in trange(200):
    v_test[2] = 1e-2 * np.sin(t/ 100.0 * 2 * np.pi)
    cobot.update(u_cmd=v_test, u_cmd_frame="ee_link", sleep=True)
cobot.stop()

There are two ways of updating the cobot, via the terminal and from code. 
We proceed with the terminal and show the results for other tools

#### changing the tool via ros launch from terminal

simply execute the command 

```
roslaunch hrr_cobot_robot tool_update.launch tool_name:="screwdriver"
```

to update the cobot to

![Screwdriver Cobot Rviz](media/screwdriver_cobot.png)

and 

```
hrr_cobot_robot
```

to update the cobot to

![Custom Vacuum Gripper Cobot Rviz](media/vacuum_cobot.png)

In [ ]:
from hr_recycler_msgs.msg import ToolType
ToolType(ToolType.SHAFT_GRINDER)

#### Update cobot URDF from code 

further we can run the launch file 

```bash
❯ roslaunch hrr_cobot_robot tool_change.launch
```

that subscribes to the ROS-parameter ``/cobot_tool_name`` and handles the remainder of the URDF update autonomously

This can be either called from code as shown below or in the dedicated ``urdf_tool_changer`` node of the ``hrr_cobot_robot`` package

In [ ]:
rospy.set_param('/cobot_tool_name', "shaftgrinder")
rospy.get_param('/cobot_tool_name')

In [9]:
rospy.get_param('/cobot_tool_name')

KeyError: '/cobot_tool_name'

In [ ]:
# ?roslaunch.parent.ROSLaunchParent
import roslaunch
import rospkg
r = rospkg.RosPack()
p = Path(r.get_path('hrr_cobot_robot')) / "launch" / "change_tool" / "tool_change.launch"
assert(p.exists()), f"cannot launch {p} -> not a real file"

In [ ]:
uuid = roslaunch.rlutil.get_or_generate_uuid(None, False)
roslaunch.configure_logging(uuid)
launch = roslaunch.parent.ROSLaunchParent(uuid, [str(p)])
launch.start()

#### Update cobot URDF via ROS

In order to ease the process, the code below outlines how the URDF can be updated from code easily.

In order to allow tool-changing, the message ``ToolType`` has been created in the ``hrr_msgs`` directory, 
which the ``urdf_tool_changer`` node listens to.

This communication is added to the ``HrrCobotHandle`` handle in versions ``>=0.1.6`` via the ``change_tool`` command
that updates the current tool and checks the current transformations from ``tf``

In [ ]:
?cobot.change_tool

In [10]:
cobot.change_tool("shaftgrinder")
cobot.T_B_C_robot

  -0.3897   -5.21e-05  0.9209   -0.08453   
  -2.578e-05  1         4.566e-05 -3.875e-06  
  -0.9209   -5.942e-06 -0.3897    0.8649    
   0         0         0         1         


In [11]:
cobot.change_tool("screwdriver")
cobot.T_B_C_robot, cobot.T_B_E_robot, cobot.tool

(  -0.3897   -5.168e-05  0.9209   -0.08453   
  -2.594e-05  1         4.515e-05 -3.917e-06  
  -0.9209   -6.293e-06 -0.3897    0.8649    
   0         0         0         1         
,
   -0.3897   -5.168e-05  0.9209   -0.08453   
  -2.594e-05  1         4.515e-05 -3.917e-06  
  -0.9209   -6.293e-06 -0.3897    0.8649    
   0         0         0         1         
,
 'screwdriver')

In [ ]:
from hrr_cobot_robot.utils import tool2tool_type
tool2tool_type(cobot.tool)

In [18]:
digital_io_controller.tool = "screwdriver"

NameError: name 'digital_io_controller' is not defined

## Use EE-control via ``cobot`` interface

The cobot interface allows to change the tool via the ``change_tool`` method.

```ipython
?cobot.change_tool
```

in addition, one can access the commands outlined above from ``cobot.tool_controller``, which returns None and prints a warning if the current tool is not set to 
one of the tools controlled via the Arduino.

The sections below outline the usage for the screwdriver / shaft grinder and vacuum gripper.

### ROS-system setup for full tool-changing utility

1. default Robot control 

    ```sh
    roslaunch hrr_cobot_robot hrr_cobot_hw.launch gripper_type:="shaftgrinder"
    ```
    
2. launch URDF changer 

    ```sh
    roslaunch hrr_cobot_robot urdf_updater.launch __ns:="hrr_cobot"
    ```

    **NOTE** this will be integrated by default in future versions
 
3. run tool-controller
 
     ```sh
     roslaunch hrr_ee_tools tool_controller.launch __ns:="hrr_cobot"
     ```
  
4. open rviz to check if ``robot_description`` is actually updated of not, i.e. add RobotStatePublisher in rviz

Using this pipeline, the URDF in rviz will be updated and the Arduino (if connected properly) will enable the tools.

#### Notes / Pitfalls

**WARNING**: ROS was invented by researchers and is thus full of pitfalls and errors. The pipeline will update the ``hrr_cobot/robot_description`` parameter, while the robot_state_publisher will check for the global ``/robot_description`` parameter. In case the pipeline does not update the robot state in rviz, please run 

```bash
❯ rosparam delete /robot_description
```

Rviz will not update the ee-models by default, you have to reload the RobotModel. Simply unchecking and rechecking the check-box in rviz is sufficient.

Note that this routine as presented here does only update the robot model and drivers.
It does not trigger the tool-changer, nor drive the robot to the dedicated poses.

For simplicity the ``None`` configuration also has a B-side connected to the tool-changer, which is physically incorrect. Keep this in mind.

The cells below will not work for all tools when run in one shot, the controller expects a switching time in between using tools of (~0.5 seconds), 
which is sufficient for real applications, where the robot needs to switch between tools first.

In [6]:
cobot.change_tool(None)

True

In [17]:
# this should throw a warning at first run
cobot.tool_controller

In [ ]:
cobot.change_tool("shaftgrinder")
cobot.tool_controller.rpm = 4000

In [ ]:
r = rospy.Rate(0.5)
cobot.change_tool("screwdriver")
cobot.tool_controller.screwdriver_program = 1
r.sleep()
cobot.tool_controller.screwdriver_start()
r.sleep()
cobot.tool_controller.screwdriver_stop()

In [13]:
cobot.change_tool("vacuum")
cobot.tool_controller.vacuum = True
rospy.sleep(1.0)
cobot.tool_controller.vacuum = False

NameError: name 'cobot' is not defined